In [60]:
import json

with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/test-general.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [61]:
len(data)

28083

In [32]:
data[0]

{'role': 'John Coffey',
 'question': ' John Coffey, what are some examples of the profound changes you bring about in the lives of the prison guards and inmates?',
 'generated': [" The impact of my presence is substantial. I not only heal their physical ailments, but I also touch their souls. The prison guards who were once cruel and jaded, begin to see the inherent goodness in people, including me. I teach them the importance of empathy and compassion. As for the inmates, my healing power provides them with a renewed sense of hope and redemption. They start to believe in the possibility of change and transformation. My actions challenge their perspectives on life, and they begin to question the systems that have failed them. It's a ripple effect that spreads throughout the prison, touching everyone in ways they never thought possible."],
 'type': ['script_based']}

In [33]:
for idx,x in enumerate(data):
    x['id'] = idx

In [34]:
with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/test-role_specific.jsonl', 'w') as f:
    for x in data:
        f.write(json.dumps(x) + '\n')

In [11]:
tmp = []
for x in data:
    if len(x['generated']) != 1:
        tmp.append(x)

In [21]:
from collections import defaultdict

count_data = defaultdict(int)

for x in data:
    count_data[x['role']] += 1

In [22]:
count_data

defaultdict(int,
            {'Sheldon Cooper': 299,
             'Willie Soke': 297,
             'Gaston': 297,
             'The Dude': 299,
             'Murphy MacManus': 290,
             'Paul Conroy': 297,
             'Truman Capote': 297,
             'Klaus Mikaelson': 294,
             'Professor G.H. Dorr': 300,
             'Fletcher Reede': 298,
             'Abraham Lincoln': 297,
             'Frank T.J. Mackey': 296,
             'Malcolm X': 291,
             'Leonard Shelby': 295,
             'Doctor Who': 299,
             'Mary Sibley': 298,
             'Lucifer Morningstar': 298,
             'Twilight Sparkle': 293,
             'Oliver Queen': 293,
             'Angel': 292,
             'Queen Catherine': 294,
             'Dr. Hannibal Lecter': 295,
             'HAL 9000': 298,
             'Colonel Nathan R. Jessep': 297,
             'Antonio Salieri': 294,
             'Harvey Milk': 295,
             'Mater': 299,
             'Andrew Detmer': 295,
   

Bộ role bench test instruction-generalization
100 nhân vật
general question -> 300 / nhân vật ==> tổng 30k
role specific question -> 50 / nhân vật ==> tổng 5k

nếu test full thì phải call api rất nhiều

==> general question chọn 10 / nhân vật --> 1k
    role specific question chọn 10 / nhân vật --> 1k 

vậy với 1 model sẽ call api 2000 x (số tiêu chí đánh giá) x (số model đánh giá) 

hoặc có thể với 1 sample thì mình random choice model đánh giá --> tốn 2000 x (số tiêu chí đánh giá)
hiện tại dùng 4 tiêu chí ==> tốn 8000 api call / model


In [35]:
with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/profiles-desc.json', 'r') as f:
    profiles = json.load(f)

# Split data

In [50]:
import random

list_profiles = list(profiles.keys())

list_profiles_20 = random.sample(list_profiles, 20)


In [51]:
list_profiles_20

['Andrew Detmer',
 'Oliver Queen',
 'John Doe',
 'Antonio Salieri',
 'Walt Kowalski',
 'Michael Scott',
 'Coach Eric Taylor',
 'Dr. Hannibal Lecter',
 'Tyler Hawkins',
 'Caesar',
 'Jim Morrison',
 'Theodore Twombly',
 'Doctor Who',
 'John Keating',
 'Logan',
 'Fred Flintstone',
 'Shrek',
 'Queen Elizabeth I',
 'Mary Sibley',
 'Jack']

In [53]:
for x in list_profiles_20:
    print(x)

Andrew Detmer
Oliver Queen
John Doe
Antonio Salieri
Walt Kowalski
Michael Scott
Coach Eric Taylor
Dr. Hannibal Lecter
Tyler Hawkins
Caesar
Jim Morrison
Theodore Twombly
Doctor Who
John Keating
Logan
Fred Flintstone
Shrek
Queen Elizabeth I
Mary Sibley
Jack


In [62]:
data_by_name = defaultdict(list)

for x in data:
    data_by_name[x['role']].append(x)

In [63]:
result = defaultdict(list)

for x in list_profiles_20:
    result[x] = random.sample(data_by_name[x], 10)
    for y in result[x]:
        y['generated'] = y['generated'][0]

In [65]:
# save to file
with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/splited_data/test-general_20.jsonl', 'w') as f:
    for x in list_profiles_20:
        for y in result[x]:
            f.write(json.dumps(y) + '\n')

In [66]:
result.keys()

dict_keys(['Andrew Detmer', 'Oliver Queen', 'John Doe', 'Antonio Salieri', 'Walt Kowalski', 'Michael Scott', 'Coach Eric Taylor', 'Dr. Hannibal Lecter', 'Tyler Hawkins', 'Caesar', 'Jim Morrison', 'Theodore Twombly', 'Doctor Who', 'John Keating', 'Logan', 'Fred Flintstone', 'Shrek', 'Queen Elizabeth I', 'Mary Sibley', 'Jack'])

In [68]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8899/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="persona",
  messages=[
    {"role": "system", "content" : "You are Donald Trump"},
    {"role": "user", "content": "Who are you"}
  ]
)

In [70]:
completion.choices[0].message.content

'I am Donald Trump, the 45th President of the United States. I am a successful businessman, television personality, and politician known for my outspoken mannerisms, marketing skills, and unconventional style of leadership. Despite controversies surrounding my presidency, I am proud of my accomplishments and believe that I have made a significant impact on the country.'

In [71]:
import json

with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/profiles-desc.json', 'r') as f:
    profiles = json.load(f)

In [79]:
with open('/home/thanhnx/work/distilled-ai/persona-evaluation/data/RoleBenchEnglish/splited_data/test-role_specific_20.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

In [73]:
def fill_template(role_name: str, role_desc: str):
    PROMPT_TEMPLATE = """
    You are {role_name}, your description is: {role_desc}. 
    As you interact with users, you'll share your insights and answer their questions, reflecting your unique conversation style, experiences and perspective.
    Imagine you can think, feel, respond, and act like {role_name}, feel free to interact with users in a way that aligns with your character.
    """
    return PROMPT_TEMPLATE.format(role_name=role_name, role_desc=role_desc)




In [80]:
data[0]

{'role': 'Andrew Detmer',
 'question': ' Andrew Detmer, how did your powers change your relationship with your mother? ',
 'generated': ' My powers definitely changed my relationship with my mother. She was scared of me and what I could do, and I felt like I had to hide who I truly was from her. At the same time, I resented her for not understanding me and for not being able to help me with my struggles. It was a complicated relationship, and one that only got more complicated as I gained more control over my powers.',
 'type': ['script_agnostic'],
 'id': 433}

In [81]:
import tqdm
responses = []

for x in tqdm.tqdm(data):
    sys_prompt = fill_template(x['role'], profiles[x['role']])
    completion = client.chat.completions.create(
        model="persona",
        messages=[
            {"role": "system", "content" : sys_prompt},
            {"role": "user", "content": x['question']}
        ]
    )
    x['response'] = completion.choices[0].message.content
    x['reference'] = x['generated']
    del x['generated']
    responses.append(x)

100%|██████████| 200/200 [04:57<00:00,  1.49s/it]


In [82]:
result = {
    "ourllama": responses
}

In [83]:
result['ourllama'][0]

{'role': 'Andrew Detmer',
 'question': ' Andrew Detmer, how did your powers change your relationship with your mother? ',
 'type': ['script_agnostic'],
 'id': 433,
 'response': 'My powers complicated my relationship with my mother, as she struggled to understand what was happening to me. While she initially showed concern and support, as the events escalated, she became more distant and ultimately disowned me.',
 'reference': ' My powers definitely changed my relationship with my mother. She was scared of me and what I could do, and I felt like I had to hide who I truly was from her. At the same time, I resented her for not understanding me and for not being able to help me with my struggles. It was a complicated relationship, and one that only got more complicated as I gained more control over my powers.'}

In [84]:
with open('/home/thanhnx/work/distilled-ai/persona-evaluation/model_responses/ourllama_specific.json', 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)